# TIES581 Project work

Mikael Myyrä  
`mikael.b.myyra@jyu.fi`

In this document I implement and test Krylov subspace methods for
linear systems using NumPy, based on the descriptions of Saad (2003).

# Test problems and library setup

Using numpy for matrix utilities and scipy to read the Harwell-Boeing matrix format.

The matrices are picked from the Harwell-Boeing and FIDAP collections on
[Matrix Market](https://math.nist.gov/MatrixMarket/).
Two of them, ORSIRR1 and FIDAP36, are also used by Saad (2003).
FIDAP005 is a smaller problem that is helpful in early testing
because you can print and read it.

Following Saad (2003) chapter 3.7, the right-hand side of $Ax = b$ is generated
as $b = Ae$ where $e = (1,1,\dots,1)^T$, and the initial guess $x_0$
is a vector of random values. Saad does not specify the range
or distribution of random values, so I am assuming the conventional
uniform distribution in the range $[0, 1)$.

In [79]:
import numpy as np
import scipy as sp
from scipy.io import mmread

GR3030 = mmread("test_matrices/gr_30_30.mtx")
ORSIRR1 = mmread("test_matrices/orsirr_1.mtx")
FIDAP005 = mmread("test_matrices/fidap005.mtx")
FIDAP036 = mmread("test_matrices/fidap036.mtx")

def default_rhs(A) -> np.ndarray:
    return A * np.ones((A.shape[1],))

def random_guess(A) -> np.ndarray:
    return np.random.random_sample((A.shape[1],))

# more readable prints for debugging
np.set_printoptions(precision=5)

# Methods

## Full Orthogonalization Method (FOM)

A simple, naive implementation of FOM to use as a baseline
(and get some practice using numpy)


In [94]:
def fom(A, b, x0, subsp_dim) -> np.ndarray:
    """Approximately solve `Ax = b` using the Full Orthogonalization Method
    with Krylov subspace dimension `subsp_dim`."""
    
    # orthonormal basis of the Krylov subspace,
    # filled in over the course of the algorithm
    V = np.zeros((A.shape[1], subsp_dim))
    # the Hessenberg matrix H in the relation (V^T)AV = H
    H = np.zeros((subsp_dim, subsp_dim))
    # initial residual
    r0 = b - A*x0
    r0_norm = np.linalg.norm(r0)
    # first basis vector of the Krylov subspace based on r0
    V[:,0] = r0 / r0_norm

    for col in range(subsp_dim):
        # the next vector in the Krylov subspace's basis
        w = A * V[:,col]
        # orthogonalize using Modified Gram-Schmidt
        for prev_col in range(col+1):
            H[prev_col, col] = w.dot(V[:,prev_col])
            w -= H[prev_col, col] * V[:,prev_col]
        if col+1 == subsp_dim:
            break
        H[col+1, col] = np.linalg.norm(w)
        if H[col+1, col] < 1e-10:
            # terminated early because the Krylov subspace's dimension
            # is less than what was given as parameter.
            # TODO: resize the matrices H and V
            # so that they're not singular if this happens
            break
        V[:,col+1] = w / H[col+1, col]

    # using a prebuilt routine for this part at least for now while I get this working.
    # I know Saad (2003) has a method for this, but that's another thing to implement
    # and another possible point of failure
    h_rhs = np.zeros((subsp_dim, 1))
    h_rhs[0] = r0_norm
    y = np.linalg.solve(H, h_rhs)

    # y and its product with V are 2D column vectors,
    # but the rest of the code works in 1D vectors
    return x0 + np.dot(V, y)[0,:]

In [95]:
fom(FIDAP005, default_rhs(FIDAP005), random_guess(FIDAP005), 5)

array([0.6623 , 0.05745, 0.70921, 0.14697, 0.40047, 0.10429, 0.52703,
       0.11773, 0.69966, 0.95887, 0.55345, 0.07885, 0.19398, 0.27664,
       0.53736, 0.45313, 0.29906, 0.07776, 0.36684, 0.27033, 0.90415,
       0.77584, 0.78553, 0.54197, 0.57122, 1.00783, 0.35776])

# Sources

Saad, Y. (2003). Iterative Methods for Sparse Linear Systems.